In [100]:
# execute to import notebook styling for tables and width etc.
from IPython.core.display import HTML
import urllib.request
response = urllib.request.urlopen('https://raw.githubusercontent.com/DataScienceUWL/DS775v2/master/ds755.css')
HTML(response.read().decode("utf-8"));

<font size=18>Lesson 06 Homework</font>

# Textbook Problem 12.2-4

Reconsider the Wyndor Glass Co. problem presented in Sec. 3.1. Management now has decided that **only one** of the two new products should be produced, and the choice is to be made on the basis of maximizing profit. Introduce auxiliary binary variables to formulate an MIP model for this new version of the problem.

(a) Introduce auxiliary binary variables to formulate a mixed BIP model for this problem. Include a picture or LaTeX of the mathematical formulation in the next cell.

<font color = "blue"> *** 4 points -  answer in cell below *** (don't delete this cell) </font>

Letting $Z$ be total profit in thousands of dollars,

Maximize $Z = 3 d + 5 w$

Subject to:

$
y_1 + y_2 = 1 \\
    \\
\begin{array}{ccccc}
 d &   &    & \leq & 4 \\
   &   & 2w & \leq & 12 \\
3d & + & 2w & \leq & 18
\end{array}
$

$d \geq 0$, $w \geq 0 \\$
$0 \leq x_i \leq y_i, \text{ for } i=1,2 \\$
$y_i \text{ binary, for } i=1, 2 \\$


b) Use Pyomo to solve this model. You can use a concrete or an abstract formulation.

<font color = "blue"> *** 6 points -  answer in cell below *** (don't delete this cell) </font>


In [101]:
# unfold for code
from pyomo.environ import *

m = ConcreteModel(name="Wyndor")

m.x1 = Var(domain = NonNegativeReals) #this makes >= 0, so no need to declare in constraints
m.x2 = Var(domain = NonNegativeReals) #this makes >= 0, so no need to declare in constraints

m.y1 = Var(domain=Boolean) #This declares y_i as binary
m.y2 = Var(domain=Boolean) #This declares y_i as binary


m.profit = Objective( expr = 3*m.x1 + 5*m.x2, sense = maximize)

# Constraints:
m.cts = ConstraintList()
m.cts.add( m.y1 + m.y2 == 1)
m.cts.add( 1 * m.x1              <= 4)
m.cts.add(              2 * m.x2  <= 12)
m.cts.add( 3 * m.x1  +  2 * m.x2  <= 18)
m.cts.add( m.x1 <= m.y1)
m.cts.add( m.x2 <= m.y2)


# Solve
solver = SolverFactory('glpk')
solver.solve(m)

import babel.numbers as numbers  # needed to display as currency
print("Maximum Profit = ",
      numbers.format_currency(1000 * m.profit(), 'USD', locale='en_US')) #in thousands, correct

print("Produce Doors." if m.y1() else "Produce Windows." )

if m.y1():
    print('Produce {:0.1f} batches of doors to maximize profit per batch'.format(m.x1()))
    
if m.y2():
    print('Produce {:0.1f} batches of windows to maximize profit per batch'.format(m.x2()))


Maximum Profit =  $5,000.00
Produce Windows.
Produce 1.0 batches of windows to maximize profit per batch


# Textbook Problem 12.3-1

The Research and Development Division of the Progressive Company has been developing four possible new product lines. Management must now make a decision as to which of these four products actually will be produced and at what levels. Therefore, an operations research study has been requested to find the most profitable product mix.

A substantial cost is associated with beginning the production of any product, as given in the first row of the following table. Management’s objective is to find the product mix that maximizes the total profit (total net revenue minus start-up costs).

<img src="images/screen-prob12_3-1.png" alt="Solution" width="350" height="150">

Let the continuous decision variables $x_1, x_2, x_3,$ and $x_4$ be the production levels of products 1, 2, 3, and 4, respectively. Management has imposed the following policy constraints on these variables:

1. No more than two of the products can be produced.

2. Either product 3 or 4 can be produced only if either product 1 or 2 is produced.

3. Either 

$$5x_1 + 3x_2 + 6x_3 + 4x_4 \leq  6,000 $$

$$\text{or}$$ 

$$4x_1 + 6x_2 + 3x_3 + 5x_4 \leq 6,000 $$.

(a) Introduce auxiliary binary variables to formulate a mixed BIP model for this problem.  Include a picture or LaTeX of the mathematical formulation in the next cell.

<font color = "blue"> *** 6 points -  answer in cell below *** (don't delete this cell) </font>

Maximize $Z = 70*x_1 + 60*x_2 + 90*x_3 + 80*x_4 - (50000*y_1 + 40000*y_2 + 70000*y_3 + 60000*y_4) $

Subject to:

$
\begin{array}{l}
y_1 + y_2 + y_3 + y_4 = 2 \\
x_3 \leq x_1 + x_2 \\
x_4 \leq x_1 + x_2 \\
5x_1 + 3x_2 + 6x_3 + 4x_4 \leq  6000 + M*y_5 \\ 
4x_1 + 6x_2 + 3x_3 + 5x_4 \leq 6000 + M*(1-y_5) \\
0 \leq x_i \leq My_i, \text{ for } i=1,2,3,4 \\
x_i \geq 0, \text{ for } i=1, 2, 3, 4  \\
y_i \text{ binary, for } i=1, 2, 3, 4 \\
\end{array}
$



(b) Use Pyomo to solve this model. Use an abstract formulation.

<font color = "blue"> *** 10 points -  answer in cell below *** (don't delete this cell) </font>

In [102]:
# unfold for code
from pyomo.environ import *

# Problem data
products = ['Product1', 'Product2', 'Product3', 'Product4']
unit_revenue = dict(zip(products, [70, 60, 90, 80]))
startup_cost = dict(zip(products, [50000, 40000, 70000, 60000]))

line_item_constraint1 = dict(zip(products, [5, 3, 6, 4]))
line_item_constraint2 = dict(zip(products, [4, 6, 3, 5]))

bigM = 1000000

num_products_to_choose = 2

# Instantiate concrete model
M = ConcreteModel(name="Production")

# Decision Variables
M.x = Var(products, domain=NonNegativeReals) #x>= 0
M.y = Var(products, domain=Boolean) #binary y/n to produce
M.y5 = Var(domain=Boolean) #for Big M, binary

# Objective:  Maximize Profit
M.profit = Objective(expr=sum((unit_revenue[pr] * M.x[pr] - startup_cost[pr] * M.y[pr]) for pr in products),
                     sense=maximize)

# Constraints:
M.constraints = ConstraintList()

# produce product only if product is chosen
for pr in products: 
    M.constraints.add(M.x[pr] <= bigM * M.y[pr])

# Must have 1/2 if have 3/4.
M.constraints.add(M.x['Product3'] <= M.x['Product2'] + M.x['Product1'])
M.constraints.add(M.x['Product4'] <= M.x['Product2'] + M.x['Product1'])

# choose 2 products
M.constraints.add(sum(M.y[pr] for pr in products) <= num_products_to_choose)


#Loop within a sum function, the "OR" for capacities
# production capacities #1
M.constraints.add(sum(line_item_constraint1[pr] * M.x[pr] for pr in products) <= 6000 + bigM * M.y5)
# production capacities #2
M.constraints.add(sum(line_item_constraint2[pr] * M.x[pr] for pr in products) <= 6000 + bigM * (1-M.y5))


# Solve
solver = SolverFactory('glpk')
solver.solve(M)

import babel.numbers as numbers  # needed to display as currency
print("Maximum Profit = ",
      numbers.format_currency( M.profit(), 'USD', locale='en_US'))

print("\nWhich products and how many:")
for pr in products:
    if bool(M.y[pr]()):
        print("Produce {} ".format(pr) + "at a {:1.2f} unit level".format(M.x[pr]() ) )
    else:
        print("Do not produce {}".format(pr) )


Maximum Profit =  $80,000.00

Which products and how many:
Do not produce Product1
Produce Product2 at a 2000.00 unit level
Do not produce Product3
Do not produce Product4


# Textbook Problem 12.4-6

Speedy Delivery provides two-day delivery service of large parcels across the United States. Each morning at each collection center, the parcels that have arrived overnight are loaded onto several trucks for delivery throughout the area. Since the competitive battlefield in this business is speed of delivery, the parcels are divided among the trucks according to their geographical destinations to minimize the average time needed to make the deliveries.

On this particular morning, the dispatcher for the Blue River Valley Collection Center, Sharon Lofton, is hard at work. Her three drivers will be arriving in less than an hour to make the day’s deliveries. There are nine parcels to be delivered, all at locations many miles apart. As usual, Sharon has loaded these locations into her computer. She is using her company’s special software package, a decision support system called Dispatcher. The first thing Dispatcher does is use these locations to generate a considerable number of attractive possible routes for the individual delivery trucks. These routes are shown in the following table (where the numbers in each column indicate the order of the deliveries), along with the estimated time required to traverse the route.

<img src="images/screen-prob12_4-6.png" alt="Solution" width="350" height="150">


Dispatcher is an interactive system that shows these routes to Sharon for her approval or modification. (For example, the computer may not know that flooding has made a particular route infeasible.) After Sharon approves these routes as attractive possibilities with reasonable time estimates, Dispatcher next formulates and solves a BIP model for selecting three routes that minimize their total time while including each delivery location on exactly one route. This morning, Sharon does approve all the routes.

(a) Formulate this BIP model.  Include a picture or LaTeX of the mathematical formulation in the next cell.

<font color = "blue"> *** 6 points -  answer in cell below *** (don't delete this cell) </font>

Minimize $Z = 6x_1 + 4x_2 + 7x_3 + 5x_4 + 4x_5 + 6x_6 + 5x_7 + 3x_8 + 7x_9 + 6x_{10} $

Subject to:

$
\begin{array}{l}
A: x_1 + x_5 + x_9 = 1 \\
B: x_2 + x_4 + x_6 + x_9 + x_{10} = 1 \\
C: x_3 + x_4 + x_7 + x_9 = 1 \\
D: x_1 + x_6 + x_8 = 1 \\
E: x_3 + x_4 + x_6 = 1 \\
F: x_2 + x_5 = 1 \\
G: x_1 + x_7 + x_8 + x_{10} = 1 \\
H: x_3 + x_5 + x_{10} = 1 \\
I: x_2 + x_4 + x_7 = 1 \\
\\
x_j \text{ binary, for } j=1, 2, 3, 4,...,10 \\
\\
\text{Three Drivers: }\sum\limits_{j=1}^{10} x_j = 3 \\
\\
\end{array}
$

(b) Use Pyomo solve this model.  Use an abstract formulation.

<font color = "blue"> *** 10 points -  answer in cell below *** (don't delete this cell) </font>

In [1]:
# Unfold for code
from pyomo.environ import *
decisions = ['Route1', 'Route2', 'Route3', 'Route4', 'Route5', 'Route6', 'Route7', 'Route8', 'Route9', 'Route10']
packages = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']
time = dict(zip(decisions, [6, 4, 7, 5, 4, 6, 5, 3, 7, 6]))

#Concrete Model
model = ConcreteModel()

#Decision Variables
model.build = Var(decisions, domain=Boolean)

#Objective
model.time = Objective(expr=sum(time[d] * model.build[d] for d in decisions), sense=minimize)

#Set up package coefficient constraints:
packages = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']
coefs = {'A':dict(zip(decisions,[1, 0, 0, 0, 1, 0, 0, 0, 1, 0])),
         'B':dict(zip(decisions,[0, 1, 0, 1, 0, 1, 0, 0, 1, 1])),
         'C':dict(zip(decisions,[0, 0, 1, 1, 0, 0, 1, 0, 1, 0])),
         'D':dict(zip(decisions,[1, 0, 0, 0, 0, 1, 0, 1, 0, 0])),
         'E':dict(zip(decisions,[0, 0, 1, 1, 0, 1, 0, 0, 0, 0])),
         'F':dict(zip(decisions,[0, 1, 0, 0, 1, 0, 0, 0, 0, 0])),
         'G':dict(zip(decisions,[1, 0, 0, 0, 0, 0, 1, 1, 0, 1])),
         'H':dict(zip(decisions,[0, 0, 1, 0, 1, 0, 0, 0, 0, 1])),
         'I':dict(zip(decisions,[0, 1, 0, 1, 0, 0, 1, 0, 0, 0]))}
rhs = dict(zip(packages,[1,1,1,1,1,1,1,1,1]))

####Constraints:
model.constraints = ConstraintList()
#Coefficients of the final outcome
for p in packages:
    model.constraints.add( expr = sum(coefs[p][d]*model.build[d] for d in decisions) == rhs[p])

#Only allow 3 and only 3 routes:
model.constraints.add(expr = sum(model.build[d] for d in decisions) == 3)

# Solve
solver = SolverFactory('glpk')
solver.solve(model)

# display solution
import babel.numbers as numbers  # needed to display as currency
print("The total time of delivery is "+ 
      numbers.format_number(model.time())+" hours.")
for d in decisions:
    print("Send Driver on {} ?  ".format(d) + ["No","Yes"][int(model.build[d]())] )


The total time of delivery is 12 hours.
Send Driver on Route1 ?  No
Send Driver on Route2 ?  No
Send Driver on Route3 ?  No
Send Driver on Route4 ?  Yes
Send Driver on Route5 ?  Yes
Send Driver on Route6 ?  No
Send Driver on Route7 ?  No
Send Driver on Route8 ?  Yes
Send Driver on Route9 ?  No
Send Driver on Route10 ?  No


# Knapsack optimization

We'll revisit the Knapsack Optimization problem from Lesson 4 and 5. This time, we'll solve it using integer programming.

Given a set of items, each with a weight and a value, use binary variables and Pyomo to determine which items to include in a collection such that the total weight is less than or equal to a given limit and the total value is as large as possible. We will start with 20 items and you need to determine the collection of items that maximizes the value and keeps the total weight less than or equal to 50. 

Use the problem data as described below:

In [104]:
#Problem Data - generate random weights and values for a knapsack problem
import numpy as np
num_items = 20 #Switch between 20/40 items
np.random.seed(seed=123)
values = np.random.randint(low=5, high=50, size=num_items)
weights = np.random.randint(low=1, high=10, size=num_items)
np.random.seed() # use system clock to reset the seed so future random numbers will appear random

Your Pyomo solution should go in the next cell.  Your code should still work if the number of items is changed to 40 or more.

<font color = "blue"> *** 8 points -  answer in cell below *** (don't delete this cell) </font>

In [105]:
from pyomo.environ import *
max_weight = 50

#Concrete Model
model = ConcreteModel(name = "Knapsack")

#Decision Variables
model.y = Var(range(num_items), domain=Boolean) #<- 0's and 1's

#Objective
model.value = Objective(expr=sum(values[y] * model.y[y] for y in model.y), sense=maximize)

####Constraints:
model.constraints = ConstraintList()

#Weights of the final outcome
model.constraints.add( expr = sum(weights[y]*model.y[y] for y in model.y) <= max_weight)

# Solve
solver = SolverFactory('glpk')
solver.solve(model)

# display solution
import babel.numbers as numbers  # needed to display as currency
print("The total value of items is $"+ 
      numbers.format_number(model.value())+".")
for y in model.y:
    print("Include item {} ?  ".format(y + 1) + ["No","Yes"][int(model.y[y]())] ) #added plus one so we don't have item zero

#Print the total weight:
myvar = 0
for each in model.y:
    myvar += model.y[each]() * weights[each]
print('Total weight of the knapsack:  {}'.format(myvar))

The total value of items is $435.
Include item 1 ?  Yes
Include item 2 ?  No
Include item 3 ?  Yes
Include item 4 ?  Yes
Include item 5 ?  No
Include item 6 ?  No
Include item 7 ?  Yes
Include item 8 ?  Yes
Include item 9 ?  Yes
Include item 10 ?  Yes
Include item 11 ?  No
Include item 12 ?  Yes
Include item 13 ?  Yes
Include item 14 ?  No
Include item 15 ?  Yes
Include item 16 ?  No
Include item 17 ?  Yes
Include item 18 ?  Yes
Include item 19 ?  Yes
Include item 20 ?  No
Total weight of the knapsack:  50.0


###Original cell, keep if needed.
from pyomo.environ import *

model.y = Var(range(20),domain=boolean) <- 0's and 1's

v1 * y1 + v2 * y2 + ...

